

BERT binary classification

Unexpected long time to train and Unexpected memory usage.

DistilBERT was successful the code was in the "binary_labeled" folder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data_path = '/content/drive/MyDrive/NLP_Workspace/Data/'

!pip install -q tensorflow_text==2.4.1

Mounted at /content/drive
     |████████████████████████████████| 3.4 MB 34.6 MB/s 
     |████████████████████████████████| 394.5 MB 42 kB/s 
     |████████████████████████████████| 14.8 MB 57.5 MB/s 
     |████████████████████████████████| 462 kB 68.3 MB/s 
     |████████████████████████████████| 132 kB 73.9 MB/s 
     |████████████████████████████████| 2.9 MB 52.7 MB/s 
     |████████████████████████████████| 3.8 MB 50.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.
pydantic 1.10.2 requires typing-extensions>=4.1.0, but you have typing-extensions 3.7.4.3 which is incompatible.
jaxlib 0.3.25+cuda11.cudnn805 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
jax 0.3.25 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
grp

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import matplotlib.pyplot as plt
import csv
import pandas as pd

Binary E vs I classification

In [ ]:
binary_data_path = '/content/drive/MyDrive/NLP_Workspace/Data/Binary_Labeled/'

col_names = ["Type", "Post"]
df = pd.read_csv(binary_data_path + "E_vs_I_data.csv", names = col_names)
df.head(5)

,Type,Post
0,1,enfp and intj moments sportscenter not top ...
1,1,What has been the most life-changing experienc...
2,1,On repeat for most of today.
3,1,May the PerC Experience immerse you.
4,1,The last thing my INFJ friend posted on his fa...


In [ ]:
df.isnull().any()    ## check for nulls ... should say False

Type    False
Post     True
dtype: bool

In [ ]:
df = df.dropna(how='any',axis=0)

In [ ]:
df.groupby("Type").describe()

Post                      
       count  unique    top   freq
Type                              
0      95752   94683    ...    103
1     326932  311998  [' ']  10905

Downsampling to handel inbalanced dataset

In [ ]:
df['Type'].value_counts()

1    326932
0     95752
Name: Type, dtype: int64

In [ ]:
df_E = df[df['Type']==0]
df_E.shape

(95752, 2)

In [ ]:
df_I = df[df['Type']==1]
df_I.shape

(326932, 2)

In [ ]:
DOWNSAMPLE = 3000
df_I_downsample = df_I.sample(DOWNSAMPLE)
df_E_downsample = df_E.sample(DOWNSAMPLE)
df_I_downsample

,Type,Post
307277,1,Tourette Syndrome? Do you have any verbal tics...
80948,1,I think ESFJ and ESFP are thought as stupid be...
145538,1,I wouldn't say it's the possibilities in and o...
219729,1,Really? I believe the way our intuition and lo...
240623,1,You know you're an INFJ when you wish you coul...
...,...,...
214436,1,That tummy ache was the feeling of good karma ...
325611,1,"Hi Guys, This has been on my mind over the pa..."
204461,1,WOW you joined a year ago and you have 5165 po...
62027,1,I'm so tired. Where am I heading? To find the ...


In [ ]:
df_balanced = pd.concat([df_I_downsample, df_E_downsample])
df_balanced.shape

(6000, 2)

In [ ]:
df_balanced['Type'].value_counts()

1    3000
0    3000
Name: Type, dtype: int64

In [ ]:
df_balanced.sample(5)

,Type,Post
305894,0,A very Ne ENFP here. I assume you know Swedi...
392603,1,Isn't a good voice the gift that keeps on givi...
401151,1,*hands cookie to flourine* ENXJ. TMLT never ...
8538,1,alakum salaam dats ok nice to hear from u
278008,0,I'd have to go with: Captain Jack Sparrow Pa...


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_balanced['Post'],df_balanced['Type'], stratify=df_balanced['Type'])


In [ ]:
y_train.head(4)

5680      0
84336     1
256318    0
30650     1
Name: Type, dtype: int64

In [ ]:
X_train


5680                  I remember Badgers being pretty cool.
84336     I rarely initiate things. Unless I'm in a situ...
256318    Don't listen to them. You own your life, you d...
30650     Okay, I'm going to give this a shot. I don't t...
124077    i definitely relate to this...... i think it g...
                                ...                        
231139    Definitely Han Solo. It always bugs me to hear...
245249    I wouldn't say that I'm a flirt, but I can eas...
3462                             I was wondering about you.
153063    Hell, I could go in depth about this, but I'll...
339472    normally near the front. i hate sitting in the...
Name: Post, Length: 4500, dtype: object

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.84351707, -0.51327276, -0.8884572 , ..., -0.74748874,
        -0.75314736,  0.91964483],
       [-0.8720836 , -0.5054398 , -0.9444668 , ..., -0.85847497,
        -0.7174535 ,  0.8808298 ]], dtype=float32)>

In [ ]:
e = get_sentence_embeding([
    "banana", 
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[3]],[e[4]])

array([[0.98720354]], dtype=float32)

Built Model

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
# sigmoid to relu

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'sequence_output':  109482241   keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [ ]:
len(X_train)

4500

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]
# 
model.compile(optimizer='SGD',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
141/141 [==============================] - 1201s 8s/step - loss: 0.7137 - accuracy: 0.4993 - precision: 0.4918 - recall: 0.4277
Epoch 2/5
141/141 [==============================] - 1217s 9s/step - loss: 0.7078 - accuracy: 0.4986 - precision: 0.4974 - recall: 0.5060
Epoch 3/5
119/141 [========================>.....] - ETA: 3:10 - loss: 0.7161 - accuracy: 0.4938 - precision: 0.4903 - recall: 0.4872

50/469 0.7323